

# MONAI

MONAI is useful whenever you deal with **medical images**, especially in formats and tasks common in hospitals and research labs (e.g., MRI, CT, PET, X-ray, Ultrasound).

```bash
pip install monai[all]
```


---

# 1. **Medical Image Segmentation** (most common use)

Examples:

* Brain tumor segmentation (MRI)
* Organ segmentation (liver, kidneys, heart, lungs)
* Vessel segmentation
* Lesion segmentation (COVID lung opacities, liver lesions)
* Breast cancer segmentation (MRI, Ultrasound)

Why MONAI helps:

* Built-in 2D/3D U-Net, UNet++, UNETR (Transformer-based)
* Medical-style augmentations (elastic deformation, intensity shifts, bias field, random cropping on label regions)
* Patch-wise training for huge 3D scans
* Metrics like Dice, Hausdorff

---

# 2. **Medical Image Classification**

Examples:

* Classify chest X-rays into normal vs pneumonia
* Detect diabetic retinopathy from fundus images
* Classify MRI sequences (T1, T2, FLAIR)
* Detect fractures in X-rays

Why MONAI helps:

* Easy data loading for DICOM/NIfTI
* Medical-oriented transforms (windowing, orientation, spacing)
* Pretrained 2D/3D models for medical images

---

# 3. **Medical Image Registration**

Examples:

* Align MRI scans of the same patient (pre-op vs post-op)
* CT–MRI multimodal registration
* Template-to-subject alignment (e.g., brain atlas)

Why MONAI helps:

* Built-in registration networks (VNet, Global/Local registration)
* SpatialTransformer layers based on VoxelMorph
* Diffeomorphic warping tools

---

# 4. **Medical Image Detection**

Examples:

* Detect lung nodules in CT
* Detect lesions in mammography
* Bounding-box detection for polyps in endoscopy

Why MONAI helps:

* Integrates with torchvision-style detectors
* Handles 3D bounding boxes
* 2D/3D detection models

---

# 5. **Medical Image Reconstruction**

Examples:

* MRI reconstruction from undersampled k-space
* CT sparse-view reconstruction
* Denoising & super-resolution for low-dose CT

Why MONAI helps:

* k-space transforms
* Physics-informed neural networks
* Integration with Total Variation losses

---

# 6. **Medical Image Preprocessing Pipelines**

Even if you don’t use MONAI for the model, you can use it for:

* Converting DICOM to tensors
* Resampling to isotropic spacing
* Normalizing Hounsfield units (HU)
* Cropping around organs
* Bias field correction
* Histogram normalization

This is a huge win compared to writing everything by hand.

---

# 7. **End-to-End Medical AI Pipelines**

Examples:

* Radiology triage system
* Automated CT lung-analysis pipeline
* Oncology tumor-tracking
* Surgical navigation
* Medical dataset cleaning and annotation tools
* Federated learning across hospitals (MONAI FL)

MONAI includes:

* MONAI Core (models, transforms, utilities)
* MONAI Label (annotation tool)
* MONAI Deploy (building medical AI apps for hospitals)

---




---

# 3. Core Concepts of MONAI

MONAI is built around these pillars:

## 3.1 Composables: `Transforms`

Transforms are **deterministic or random operations composed into a pipeline**:

Example:

```python
train_transforms = monai.transforms.Compose([
    monai.transforms.LoadImaged(keys=["image", "label"]),
    monai.transforms.AddChanneld(keys=["image", "label"]),
    monai.transforms.ScaleIntensityd(keys=["image"]),
    monai.transforms.RandSpatialCropd(keys=["image", "label"], roi_size=(96,96,96), random_size=False),
    monai.transforms.RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
    monai.transforms.ToTensord(keys=["image", "label"])
])
```

These work naturally with **medical formats** (nii.gz, mhd, nrrd, dicom).

---

## 3.2 Dataset types

MONAI provides efficient dataset implementations:

### **`CacheDataset`**

* Caches transformed samples in memory
* Faster training but uses RAM

### **`SmartCacheDataset`**

* Rotates cached items and improves RAM usage

### **`Dataset`**

* Basic, no caching

Example:

```python
train_ds = monai.data.CacheDataset(
    data=train_files, 
    transform=train_transforms,
    cache_rate=1.0      # fully cache
)
```

---

## 3.3 DataLoader (PyTorch)

MONAI integrates with PyTorch DataLoader:

```python
train_loader = monai.data.DataLoader(
    train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=4
)
```

---

## 3.4 Models (Networks)

Example UNet:

```python
from monai.networks.nets import UNet

model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128),
    strides=(2, 2, 2),
)
```

Other models available:

* `UNETR`
* `SwinUNETR`
* `SegResNet`
* `DynUNet`
* `DenseNet`
* `ViT`

---

## 3.5 Losses

Designed for segmentation:

```python
from monai.losses import DiceLoss, DiceCELoss

loss_fn = DiceCELoss(to_onehot_y=True, softmax=True)
```

---

## 3.6 Metrics

Measuring segmentation quality:

```python
from monai.metrics import DiceMetric

dice_metric = DiceMetric(include_background=False, reduction="mean")
```

Internally, MONAI computes

$$
\text{Dice} = \frac{2 |\text{Prediction} \cap \text{GroundTruth}|}{|\text{Prediction}| + |\text{GroundTruth}|}.
$$

---

---

## 4.2 Transforms

```python
from monai.transforms import (
    LoadImaged, AddChanneld, ScaleIntensityd,
    RandFlipd, RandRotate90d, ToTensord, Compose
)

train_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    AddChanneld(keys=["image", "label"]),
    ScaleIntensityd(keys=["image"]),
    RandRotate90d(keys=["image", "label"], prob=0.5, max_k=3),
    ToTensord(keys=["image", "label"])
])
```

---

## 4.3 Dataset + DataLoader

```python
from monai.data import CacheDataset, DataLoader

train_ds = CacheDataset(train_files, train_transforms, cache_rate=1.0)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=2)
```

---

## 4.4 Model, Loss, Optimizer

```python
from monai.networks.nets import UNet
from monai.losses import DiceCELoss
import torch

device = "cuda"

model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128),
    strides=(2, 2, 2)
).to(device)

loss_fn = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
```

---

## 4.5 Training loop (clean version)

```python
for epoch in range(10):
    model.train()
    running_loss = 0

    for batch in train_loader:
        img = batch["image"].to(device)
        lab = batch["label"].to(device)

        optimizer.zero_grad()
        pred = model(img)
        loss = loss_fn(pred, lab)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch}, Loss = {running_loss / len(train_loader)}")
```

---

# 5. How MONAI Differs From Standard PyTorch

| Aspect                 | PyTorch        | MONAI                                         |
| ---------------------- | -------------- | --------------------------------------------- |
| Medical image IO       | None           | Yes (nifti, dicom, mhd, etc.)                 |
| Complex transforms     | Manual         | Huge library (affine, spatial crop, patching) |
| 3D UNets               | Write manually | Many pretrained models                        |
| Patch-based training   | Manual         | Automated                                     |
| Caching / SmartCache   | No             | Yes                                           |
| Reproducible workflows | Must assemble  | Included patterns                             |

---

# 6. Additional Useful Blocks

## 6.1 Sliding window inference

For big CT/MRI volumes:

```python
from monai.inferers import sliding_window_inference

pred = sliding_window_inference(
    inputs=img,
    roi_size=(96, 96, 96),
    sw_batch_size=4,
    predictor=model
)
```

---

## 6.2 Save NIfTI results

```python
from monai.data import write_nifti

write_nifti(pred.cpu(), "prediction.nii.gz", affine=np.eye(4))
```

---



# Rule of Thumb

* **Use MONAI losses, metrics, transforms** → because they are designed for medical segmentation and often outperform raw PyTorch equivalents.
* **Use PyTorch optimizers, schedulers, AMP, gradients** → because MONAI does *not* replace these and PyTorch versions are standard.

Below is the clean breakdown.

---

# 1. LOSS FUNCTIONS

### Use MONAI versions (recommended)

✔ Specialized for segmentation
✔ Include softmax, one-hot, class weighting
✔ Numerically stable
✔ Handle 3D, multi-class, patch-based workflows

Examples:

```python
from monai.losses import DiceLoss, DiceCELoss, FocalLoss
```

Why not plain PyTorch loss?

PyTorch `CrossEntropyLoss` is fine for classification, but for segmentation you often need:

* Dice
* Dice + CE
* Focal
* Boundary losses
* Generalized Dice

These are **not** in PyTorch.

---

# 2. OPTIMIZERS

### Use standard PyTorch optimizers

MONAI does **not** introduce optimizers.

Example:

```python
import torch
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
```

This is the correct choice.

---

# 3. SCHEDULERS

### Use PyTorch schedulers

```python
from torch.optim.lr_scheduler import CosineAnnealingLR
scheduler = CosineAnnealingLR(optimizer, T_max=100)
```

MONAI does not replace schedulers.

---

# 4. BACKPROP, AMP, GRAD-SCALING

### Use PyTorch AMP

```python
from torch.amp import autocast, GradScaler
```

MONAI integrates with it but does not replace it.

---

# 5. METRICS

### Use MONAI metrics for segmentation, classification, detection

Examples:

```python
from monai.metrics import DiceMetric
from monai.metrics import HausdorffDistanceMetric
from monai.metrics import ConfusionMatrixMetric
```

PyTorch does not provide medical segmentation metrics.

---

# 6. MODELS

### Use MONAI models

Already covered earlier.

---

# 7. TRANSFORMS / DATASETS

### Use MONAI

PyTorch does not understand medical data formats (Nifti, DICOM, orientation, spacing, patch sampling).

---

# 8. FULL TRAINING LOOP — CORRECT MIX

Here is the **ideal combination** (this is what almost all MONAI papers use):

```python
import torch
from monai.networks.nets import UNet
from monai.losses import DiceCELoss
from monai.metrics import DiceMetric
from monai.transforms import Compose, LoadImaged, AddChanneld
from monai.data import CacheDataset, DataLoader

model = UNet(
    spatial_dims=3, in_channels=1, out_channels=2,
    channels=(16,32,64,128), strides=(2,2,2)
).to("cuda")

loss_fn = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

dice_metric = DiceMetric(include_background=False)

# inside training loop
loss = loss_fn(pred, label)
loss.backward()
optimizer.step()
```

This is the **standard MONAI workflow**:

* MONAI loss
* MONAI metrics
* PyTorch optimizer
* PyTorch gradient updates
* MONAI transforms
* MONAI datasets
* MONAI models

---

# 9. Why MONAI Doesn't Replace Optimizers / Schedulers

Because:

* Medical imaging research uses **standard PyTorch optimizers**
* Community wants compatibility with existing PyTorch training code
* No real advantage in re-implementing optimizers

So MONAI only extends the parts specifically needed for medical imaging.

---

# 10. Quick Summary Table

| Component        | Use PyTorch? | Use MONAI?    | Notes                          |
| ---------------- | ------------ | ------------- | ------------------------------ |
| Loss             | ✔ optional   | ✔ recommended | Use MONAI for segmentation     |
| Optimizer        | ✔            | ❌             | Always use PyTorch             |
| Scheduler        | ✔            | ❌             | Standard PyTorch               |
| AMP / GradScaler | ✔            | ❌             | From PyTorch                   |
| Transforms       | ❌            | ✔             | MONAI only                     |
| Metrics          | ❌            | ✔             | MONAI medical metrics          |
| Models           | ❌            | ✔             | MONAI UNet / UNETR / SwinUNETR |
| Datasets         | ❌            | ✔             | CacheDataset, SmartCache       |
| Inferers         | ❌            | ✔             | sliding window inference       |

---
